In [1]:
import pandas as pd
import boto3
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
data = pd.read_csv(data_url, sep=';')


In [3]:
data['quality_label'] = pd.cut(data['quality'], bins=[0, 4, 7, 10], labels=['Low', 'Average', 'High'])


In [4]:
train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data['quality_label'])
validation, test = train_test_split(test, test_size=0.5, random_state=42, stratify=test['quality_label'])


In [5]:
X_train = train.drop(['quality', 'quality_label'], axis=1)
y_train = train['quality_label']
X_validation = validation.drop(['quality', 'quality_label'], axis=1)
y_validation = validation['quality_label']
X_test = test.drop(['quality', 'quality_label'], axis=1)
y_test = test['quality_label']

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

In [7]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [8]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train)

SVC(kernel='linear', random_state=42)

In [9]:
y_test_pred_knn = knn_model.predict(X_test_scaled)
y_test_pred_svm = svm_model.predict(X_test_scaled)

In [10]:
results = {
    'K-Nearest Neighbors': {
        'Accuracy': accuracy_score(y_test, y_test_pred_knn),
        'Precision': precision_score(y_test, y_test_pred_knn, average='macro', zero_division=1),
        'Recall': recall_score(y_test, y_test_pred_knn, average='macro')
    },
    'Support Vector Machine': {
        'Accuracy': accuracy_score(y_test, y_test_pred_svm),
        'Precision': precision_score(y_test, y_test_pred_svm, average='macro', zero_division=1),
        'Recall': recall_score(y_test, y_test_pred_svm, average='macro')
    }
}

In [11]:
results

{'K-Nearest Neighbors': {'Accuracy': 0.9387755102040817,
  'Precision': 0.8192667862123929,
  'Recall': 0.49779735682819376},
 'Support Vector Machine': {'Accuracy': 0.926530612244898,
  'Precision': 0.9755102040816327,
  'Recall': 0.3333333333333333}}

In [12]:
s3 = boto3.client('s3')
bucket_name = 'p4-visruta'  
key = 'results_data.json'  

# Upload the results to S3
s3.put_object(Bucket=bucket_name, Key=key, Body=json.dumps(results).encode('utf-8'))

{'ResponseMetadata': {'RequestId': 'EAM69SJBQ6QR2VFB',
  'HostId': 'W1+ONjJN0h0suF0WMivxQGK//ywe2miUEla8/RiNQpjB4NbxOxSNkh9Z2o3z5xkqqLsVRojtsUIkaHthCrPJRA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'W1+ONjJN0h0suF0WMivxQGK//ywe2miUEla8/RiNQpjB4NbxOxSNkh9Z2o3z5xkqqLsVRojtsUIkaHthCrPJRA==',
   'x-amz-request-id': 'EAM69SJBQ6QR2VFB',
   'date': 'Fri, 26 Apr 2024 04:26:54 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"2c3a5d9a543de9aff8c1f68a2131bbed"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2c3a5d9a543de9aff8c1f68a2131bbed"',
 'ServerSideEncryption': 'AES256'}